<center>
<img src="../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
<center>
Автор материала: Юрий Кашницкий, программист-исследователь Mail.Ru Group <br> 

Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание № 10 (демо)
## <center> Прогнозирование задержек вылетов

Ваша задача – побить единственный бенчмарк в [соревновании](https://www.kaggle.com/c/flight-delays-2017) на Kaggle Inclass. Подробных инструкций не будет, будет только тезисно описано, как получен этот бенчмарк. Конечно, с помощью Xgboost. Надеюсь, на данном этапе курса вам достаточно бросить полтора взгляда на данные, чтоб понять, что это тот тип задачи, в которой затащит Xgboost. Но проверьте еще Catboost.

In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [2]:
train = pd.read_csv('../data/flight_delays_train.csv')
test = pd.read_csv('../data/flight_delays_test.csv')

In [3]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [4]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


Итак, надо по времени вылета самолета, коду авиакомпании-перевозчика, месту вылета и прилета и расстоянию между аэропортами вылета и прилета предсказать задержку вылета более 15 минут. В качестве простейшего бенчмарка возьмем логистическую регрессию и два признака, которые проще всего взять: `DepTime` и `Distance`. У такой модели результат – 0.68202 на LB. 

In [5]:
X_train, y_train = train[['Distance', 'DepTime']].values, train['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test[['Distance', 'DepTime']].values

X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, 
                                                                y_train, 
                                                                test_size=0.3, 
                                                                random_state=17)

In [6]:
logit = LogisticRegression(random_state=17)

logit.fit(X_train_part, y_train_part)
logit_valid_pred = logit.predict_proba(X_valid)[:, 1]

roc_auc_score(y_valid, logit_valid_pred)

0.6789733731013721

In [10]:
logit.fit(X_train, y_train)
logit_test_pred = logit.predict_proba(X_test)[:, 1]

pd.Series(logit_test_pred, 
          name='dep_delayed_15min').to_csv('../data/logit_2feat.csv', 
                                           index_label='id', 
                                           header=True)

Как был получен бенчмарк в соревновании:
- Признаки `Distance` и  `DepTime` брались без изменений
- Создан признак "маршрут" из исходных `Origin` и `Dest`
- К признакам `Month`, `DayofMonth`, `DayOfWeek`, `UniqueCarrier` и "маршрут" применено OHE-преобразование (`LabelBinarizer`)
- Выделена отложенная выборка
- Обучалась логистическая регрессия и градиентный бустинг (xgboost), гиперпараметры бустинга настраивались на кросс-валидации, сначала те, что отвечают за сложность модели, затем число деревьев фиксировалось равным 500 и настраивался шаг градиентного спуска
- С помощью `cross_val_predict` делались прогнозы обеих моделей на кросс-валидации (именно предсказанные вероятности), настраивалась линейная смесь ответов логистической регрессии и градиентного бустинга вида $w_1 * p_{logit} + (1 - w_1) * p_{xgb}$, где $p_{logit}$ – предсказанные логистической регрессией вероятности класса 1, $p_{xgb}$ – аналогично. Вес $w_1$ подбирался вручную. 
- В качестве ответа для тестовой выборки бралась аналогичная комбинация ответов двух моделей, но уже обученных на всей обучающей выборке.

Описанный план ни к чему не обязывает – это просто то, как решение получил автор задания. Возможно, мы не захотите следовать намеченному плану, а добавите, скажем, пару хороших признаков и обучите лес из тысячи деревьев.

Удачи!

In [3]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import cross_val_predict
from tqdm import tqdm_notebook
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

import warnings
warnings.simplefilter('ignore')

## Обработка признаков

In [383]:
train = pd.read_csv('../data/flight_delays_train.csv')
test = pd.read_csv('../data/flight_delays_test.csv')

In [335]:
#train['Route'] = train['Origin'] + '_' + train['Dest']
#test['Route'] = test['Origin'] + '_' + test['Dest']
#
#train = train.drop(['Origin', 'Dest'], axis=1);
#test = test.drop(['Origin', 'Dest'], axis=1);

In [4]:
train['Holidays'] = train['DayOfWeek'].copy().apply(lambda x: 1 if int(x[2:]) > 5 else 0)
test['Holidays'] = test['DayOfWeek'].copy().apply(lambda x: 1 if int(x[2:]) > 5 else 0)

In [337]:
def MultiLabelBinarizer(train, test, features):
    lb = LabelBinarizer()
    
    for feature in tqdm_notebook(features):
        lb.fit(train[feature])
        train_transformed = lb.transform(train[feature])
        test_transformed = lb.transform(test[feature])

        for i in range(train_transformed.shape[1]):
            train[feature + '_' + str(i + 1)] = train_transformed.T[i]
            test[feature + '_' + str(i + 1)] = test_transformed.T[i]

        train = train.drop([feature], axis=1);
        test = test.drop([feature], axis=1);
        
    return (train, test)

In [338]:
train.shape, test.shape

((100000, 10), (100000, 9))

In [339]:
train, test = MultiLabelBinarizer(train, test, ['Month', 'DayofMonth', 'DayOfWeek', 
                                                'UniqueCarrier', 'Dest', 'Origin']) #, 'Route'

In [340]:
train.shape, test.shape

((100000, 654), (100000, 653))

In [341]:
train.head()

,DepTime,Distance,dep_delayed_15min,Holidays,Month_1,Month_2,Month_3,Month_4,Month_5,Month_6,...,Origin_280,Origin_281,Origin_282,Origin_283,Origin_284,Origin_285,Origin_286,Origin_287,Origin_288,Origin_289
0,1934,732,N,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1548,834,N,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1422,416,N,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1015,872,N,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1828,423,Y,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X_train, y_train = train.drop(['dep_delayed_15min'], axis=1).values, train['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=17)

## Обучение логистической регрессии

In [343]:
logit = LogisticRegression(random_state=17)

logit.fit(X_train, y_train)
logit_valid_pred = logit.predict_proba(X_valid)[:, 1]

roc_auc_score(y_valid, logit_valid_pred)

0.7035066901377162

In [344]:
params = { 
    'C': [10 ** i for i in range(-5, 5)]
}
grid = GridSearchCV(logit, params)
grid.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=17, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [345]:
grid.best_params_

{'C': 0.1}

In [346]:
roc_auc_score(y_valid, grid.best_estimator_.predict_proba(X_valid)[:, 1])

0.7037397129033889

## Обучение XGBoost

In [347]:
from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [348]:
def score(params):
    from sklearn.metrics import log_loss
    print("Training with params:", params)
    
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_valid, label=y_valid)
    
    model = xgb.train(params, dtrain) 
    predictions = model.predict(dvalid) 
    score = roc_auc_score(y_valid, predictions)
    
    print("\tScore {0}\n\n".format(score))
    return {'loss': 1 - score, 'status': STATUS_OK}

In [349]:
def optimize(trials):
    space = {
        'learning_rate': hp.quniform('learning_rate', 0.001, 100, 0.001),
        'n_estimators': hp.quniform('n_estimators', 100, 1000, 1),
        'eta': hp.quniform('eta', 0.025, 0.5, 0.025),
        'max_depth':  hp.choice('max_depth', np.arange(1, 14, dtype=int)),
        'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
        'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
        'gamma': hp.quniform('gamma', 0.5, 1, 0.05),
        'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
        'eval_metric': 'auc',
        'objective': 'binary:logistic',
        'nthread': 4,
        'booster': 'gbtree',
        'tree_method': 'exact',
        'silent': 1,
    }
    
    best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=250)
    return best

In [350]:
trials = Trials()
best_params = optimize(trials)
best_params

Training with params: {'booster': 'gbtree', 'colsample_bytree': 0.75, 'eta': 0.45, 'eval_metric': 'auc', 'gamma': 0.7000000000000001, 'learning_rate': 39.699, 'max_depth': 7, 'min_child_weight': 5.0, 'n_estimators': 852.0, 'nthread': 4, 'objective': 'binary:logistic', 'silent': 1, 'subsample': 0.8500000000000001, 'tree_method': 'exact'}
	Score 0.529447704346019


Training with params: {'booster': 'gbtree', 'colsample_bytree': 0.65, 'eta': 0.35000000000000003, 'eval_metric': 'auc', 'gamma': 0.8, 'learning_rate': 98.34, 'max_depth': 10, 'min_child_weight': 2.0, 'n_estimators': 952.0, 'nthread': 4, 'objective': 'binary:logistic', 'silent': 1, 'subsample': 0.6000000000000001, 'tree_method': 'exact'}
	Score 0.5


Training with params: {'booster': 'gbtree', 'colsample_bytree': 0.8, 'eta': 0.4, 'eval_metric': 'auc', 'gamma': 0.8500000000000001, 'learning_rate': 43.388, 'max_depth': 9, 'min_child_weight': 5.0, 'n_estimators': 158.0, 'nthread': 4, 'objective': 'binary:logistic', 'silent': 1, 's


Training with params: {'booster': 'gbtree', 'colsample_bytree': 0.8, 'eta': 0.375, 'eval_metric': 'auc', 'gamma': 0.7000000000000001, 'learning_rate': 32.461, 'max_depth': 8, 'min_child_weight': 6.0, 'n_estimators': 330.0, 'nthread': 4, 'objective': 'binary:logistic', 'silent': 1, 'subsample': 0.7000000000000001, 'tree_method': 'exact'}
	Score 0.46475347509416864


Training with params: {'booster': 'gbtree', 'colsample_bytree': 0.9500000000000001, 'eta': 0.05, 'eval_metric': 'auc', 'gamma': 0.8500000000000001, 'learning_rate': 25.573, 'max_depth': 9, 'min_child_weight': 2.0, 'n_estimators': 279.0, 'nthread': 4, 'objective': 'binary:logistic', 'silent': 1, 'subsample': 0.6000000000000001, 'tree_method': 'exact'}
	Score 0.494876814165904


Training with params: {'booster': 'gbtree', 'colsample_bytree': 0.75, 'eta': 0.2, 'eval_metric': 'auc', 'gamma': 0.9500000000000001, 'learning_rate': 6.239, 'max_depth': 13, 'min_child_weight': 4.0, 'n_estimators': 389.0, 'nthread': 4, 'objective': 'b

Training with params: {'booster': 'gbtree', 'colsample_bytree': 0.65, 'eta': 0.1, 'eval_metric': 'auc', 'gamma': 0.5, 'learning_rate': 30.186, 'max_depth': 7, 'min_child_weight': 1.0, 'n_estimators': 861.0, 'nthread': 4, 'objective': 'binary:logistic', 'silent': 1, 'subsample': 0.65, 'tree_method': 'exact'}
	Score 0.5


Training with params: {'booster': 'gbtree', 'colsample_bytree': 0.8500000000000001, 'eta': 0.225, 'eval_metric': 'auc', 'gamma': 0.7000000000000001, 'learning_rate': 83.76, 'max_depth': 4, 'min_child_weight': 4.0, 'n_estimators': 701.0, 'nthread': 4, 'objective': 'binary:logistic', 'silent': 1, 'subsample': 0.75, 'tree_method': 'exact'}
	Score 0.6214383338190391


Training with params: {'booster': 'gbtree', 'colsample_bytree': 0.75, 'eta': 0.17500000000000002, 'eval_metric': 'auc', 'gamma': 0.65, 'learning_rate': 11.625, 'max_depth': 12, 'min_child_weight': 3.0, 'n_estimators': 982.0, 'nthread': 4, 'objective': 'binary:logistic', 'silent': 1, 'subsample': 0.8, 'tree_met

Training with params: {'booster': 'gbtree', 'colsample_bytree': 0.8500000000000001, 'eta': 0.30000000000000004, 'eval_metric': 'auc', 'gamma': 0.9500000000000001, 'learning_rate': 13.52, 'max_depth': 1, 'min_child_weight': 5.0, 'n_estimators': 595.0, 'nthread': 4, 'objective': 'binary:logistic', 'silent': 1, 'subsample': 0.9, 'tree_method': 'exact'}
	Score 0.6302174650649919


Training with params: {'booster': 'gbtree', 'colsample_bytree': 0.6000000000000001, 'eta': 0.275, 'eval_metric': 'auc', 'gamma': 0.8500000000000001, 'learning_rate': 16.741, 'max_depth': 11, 'min_child_weight': 1.0, 'n_estimators': 652.0, 'nthread': 4, 'objective': 'binary:logistic', 'silent': 1, 'subsample': 0.65, 'tree_method': 'exact'}
	Score 0.5030326907568324


Training with params: {'booster': 'gbtree', 'colsample_bytree': 1.0, 'eta': 0.45, 'eval_metric': 'auc', 'gamma': 0.8500000000000001, 'learning_rate': 51.476, 'max_depth': 10, 'min_child_weight': 6.0, 'n_estimators': 426.0, 'nthread': 4, 'objective': '

Training with params: {'booster': 'gbtree', 'colsample_bytree': 0.8500000000000001, 'eta': 0.325, 'eval_metric': 'auc', 'gamma': 0.7000000000000001, 'learning_rate': 22.493000000000002, 'max_depth': 11, 'min_child_weight': 3.0, 'n_estimators': 753.0, 'nthread': 4, 'objective': 'binary:logistic', 'silent': 1, 'subsample': 0.55, 'tree_method': 'exact'}
	Score 0.5


Training with params: {'booster': 'gbtree', 'colsample_bytree': 0.9500000000000001, 'eta': 0.1, 'eval_metric': 'auc', 'gamma': 0.6000000000000001, 'learning_rate': 27.392, 'max_depth': 6, 'min_child_weight': 2.0, 'n_estimators': 784.0, 'nthread': 4, 'objective': 'binary:logistic', 'silent': 1, 'subsample': 0.65, 'tree_method': 'exact'}
	Score 0.4998875435208497


Training with params: {'booster': 'gbtree', 'colsample_bytree': 0.8, 'eta': 0.4, 'eval_metric': 'auc', 'gamma': 0.75, 'learning_rate': 15.359, 'max_depth': 12, 'min_child_weight': 4.0, 'n_estimators': 938.0, 'nthread': 4, 'objective': 'binary:logistic', 'silent': 1, '

Training with params: {'booster': 'gbtree', 'colsample_bytree': 0.8, 'eta': 0.05, 'eval_metric': 'auc', 'gamma': 0.8500000000000001, 'learning_rate': 50.475, 'max_depth': 11, 'min_child_weight': 4.0, 'n_estimators': 754.0, 'nthread': 4, 'objective': 'binary:logistic', 'silent': 1, 'subsample': 0.9500000000000001, 'tree_method': 'exact'}
	Score 0.5


Training with params: {'booster': 'gbtree', 'colsample_bytree': 0.8500000000000001, 'eta': 0.17500000000000002, 'eval_metric': 'auc', 'gamma': 0.9, 'learning_rate': 1.782, 'max_depth': 10, 'min_child_weight': 3.0, 'n_estimators': 857.0, 'nthread': 4, 'objective': 'binary:logistic', 'silent': 1, 'subsample': 1.0, 'tree_method': 'exact'}
	Score 0.6672959250938151


Training with params: {'booster': 'gbtree', 'colsample_bytree': 0.6000000000000001, 'eta': 0.15000000000000002, 'eval_metric': 'auc', 'gamma': 1.0, 'learning_rate': 3.902, 'max_depth': 1, 'min_child_weight': 4.0, 'n_estimators': 793.0, 'nthread': 4, 'objective': 'binary:logistic', 

{'colsample_bytree': 0.75,
 'eta': 0.17500000000000002,
 'gamma': 0.7000000000000001,
 'learning_rate': 0.23800000000000002,
 'max_depth': 11,
 'min_child_weight': 3.0,
 'n_estimators': 937.0,
 'subsample': 0.7000000000000001}

In [351]:
params = {
    'colsample_bytree': 0.75,
    'eta': 0.175,
    'gamma': 0.7,
    'learning_rate': 0.238,
    'max_depth': 11,
    'min_child_weight': 3.0,
    'n_estimators': 937.0,
    'subsample': 0.7,
    'eval_metric': 'auc',
    'objective': 'binary:logistic',
    'nthread': 4,
    'booster': 'gbtree',
    'tree_method': 'exact',
    'silent': 1,
}

dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_valid, label=y_valid)
    
xgb_classifier = xgb.train(params, dtrain) 
xgb_valid_pred = xgb_classifier.predict(dvalid) 
xgb_score = roc_auc_score(y_valid, xgb_valid_pred)
xgb_score

0.7254448506803479

In [357]:
best_w = 0;
best_score = 0;

for w in np.arange(0.25, 0.26, 0.00001):
    pred = w * logit_valid_pred + (1 - w) * xgb_valid_pred
    score = roc_auc_score(y_valid, pred)
    
    if score > best_score:
        best_score = score
        best_w = w

print("Best Score %f\nBest w %f" % (best_score, best_w))

Best Score 0.728474
Best w 0.255200


## Predict on train + valid

In [370]:
X_train.shape, X_valid.shape

((70000, 653), (30000, 653))

In [18]:
X = np.vstack([X_train, X_valid])
y = np.hstack([y_train, y_valid])
X.shape, y.shape

((100000, 9), (100000,))

In [374]:
logit = LogisticRegression(C=0.1, random_state=17)
logit.fit(X, y)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=17, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [375]:
dmatrix = xgb.DMatrix(X, label=y)
xgb_classifier = xgb.train(params, dmatrix)

In [378]:
w = 0.255200

test.columns=['f' + str(i) for i in range(653)]
dtest = xgb.DMatrix(test)
prediction = w * logit.predict_proba(test)[:, 1] + (1 - w) * xgb_classifier.predict(dtest)

pd.Series(prediction, 
          name='dep_delayed_15min').to_csv('../data/logit + xgb + ohe + 1feat.csv', 
                                           index_label='id', 
                                           header=True)

### Score on kaggle: 0.72485

## CatBoost

In [12]:
from catboost import CatBoostClassifier, Pool

In [7]:
model = CatBoostClassifier(
    random_seed=42,
    logging_level='Silent'
)

In [14]:
categorical_features_indices = [0, 1, 2, 4, 5, 6]

In [8]:
model.fit(
    X_train, y_train,
    cat_features=categorical_features_indices
);

Learning rate set to 0.055755


In [10]:
roc_auc_score(y_valid, model.predict_proba(X_valid)[:, 1])

0.7555835171429395

In [19]:
model.fit(
    X, y,
    cat_features=categorical_features_indices
);

prediction = model.predict_proba(test)[:, 1]
pd.Series(prediction, 
          name='dep_delayed_15min').to_csv('../data/catboost + 1feat.csv', 
                                           index_label='id', 
                                           header=True)

### Score on kaggle: 0.73196